# Mario Kart 8 Deluxe World Record Pandas Dataframe Creation

Mario Kart 8 Deluxe is a [best selling video game](https://en.wikipedia.org/wiki/List_of_best-selling_video_games#List) for the [Nintendo Switch video game console](https://en.wikipedia.org/wiki/Nintendo_Switch). It is a car racing game with over 100 different race tracks.

The website [mkwrs.com](https://mkwrs.com/mk8dx/) contains a list of the world records for each of the video games tracks. This code translates the HTML table from the website into a Pandas dataframe containing all the information about each world record. Each entry in the dataframe contains the following information (listed in order of columns).
 - track: The name of the race track
 - cc: The speed of the cart in the record. There are two options: 150cc and 200cc. 200cc is faster than 150cc.
 - time: The time of the world record.
 - player: The player owning the record.
 - nation: The nationality of the record holder
 - date: The date the record was set
 - duration
 - character: What character was used. It should be noted that the combination of character, vehicle, tires, and glider determine the stats of the kart used to set the record.
 - vehicle: What vehicle was used.
 - tires: What tires were used.
 - glider: What glider was used.
 - splits: The splittimes for each lap.
 - mushroom: How many mushrooms are used on each lap. At the beginning of the race the player is provided 3 mushrooms which can be used at any time to provide a temporary speed boost.
 - coin: How many coins were gathered during each lap. The more coins a player has the faster their kart's top speed is. A maximum of 10 coins can be collected.
 - hybrid_used: Denotes whether hybrid controls were used. On some tracks it is advantageous to pause the game and switch controllers. This denotes if that occurred.

In [1]:
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
# open the html file with the records and load into the HTML parser. This should be replaced with code fetching from the website itself and then extracting the table.
with open('records.html') as f:
    html_text = f.read()
soup = BeautifulSoup(html_text,'html.parser')

### Parse Page

In [3]:
# this class stores all the information about each world record
class Record():
    def __init__(self, track, cc, time, player, nation, date, duration, character, vehicle,tires, glider, splits, mushroom, coin, hybrid_used):
        self.track = track
        self.cc = cc
        self.time = time
        self.player = player
        self.nation = nation
        self.date = date
        self.duration = duration
        self.character = character
        self.vehicle = vehicle
        self.tires = tires
        self.glider = glider
        self.splits = splits
        self.mushroom = mushroom
        self.coin = coin
        self.hybrid_used = hybrid_used
    def __str__(self):
        return f'{self.track} {self.cc} {self.time} {self.player} {self.nation} {self.date} {self.duration} {self.character} {self.vehicle} {self.tires} {self.glider} {self.splits} {self.mushroom} {self.coin} {self.hybrid_used} '

In [4]:
# the text on the website containing the splits, mushroom use, and coin gathered is rather complicated. This function, along with its private functions parse that text.
def parse_mouseover(mouseover_text):
    def parse_splits(splits_list):
        for i, split in enumerate(splits_list):
            splits_list[i] = float(split[2:-1])
        return splits_list
    def parse_mushrooms(mushrooms_list):
        for i, mushroom in enumerate(mushrooms_list):
            if i == 0:
                mushrooms_list[i] = int(mushroom[2])
            elif i == len(mushrooms_list)-1:
                mushrooms_list[i] = int(mushroom[0])
            else:
                mushrooms_list[i] = int(mushroom)
        return mushrooms_list
    def parse_coins(coins_list):
        for i, coin in enumerate(coins_list):
            if i == 0:
                coins_list[i] = int(coin[1])
            elif i == len(coins_list)-1:
                coins_list[i] = int(coin[0])
            else:
                coins_list[i] = int(coin[0])
        return coins_list
    data = mouseover_text.split('(')[1].split(',')[1:]
    splits_list = data[:-2]
    splits = parse_splits(splits_list)
    mushrooms_list = data[-1].split('-')
    mushrooms = parse_mushrooms(mushrooms_list)
    coins_list = data[-2].strip().split('-')
    coins = parse_coins(coins_list)
    return splits, mushrooms,coins

# this parses the text containing the time
def parse_time(time_text):
    time = time_text.strip()
    minutes, not_minutes = time.split('\'')
    minutes = int(minutes)
    seconds, milliseconds = not_minutes.split('"')
    seconds = int(seconds)
    milliseconds = int(milliseconds)
    return minutes*60+seconds+milliseconds/1000

In [5]:
records = list()
rows = soup.find_all('tr')
# some courses have multiple people with the same world records. This records how many records there are for each course. If a course is not listed it has two times
tied_times = {
    'Mario Circuit': 3,
    'GCN Baby Park': 3,
    'Big Blue': 3,
    'Tour Sydney Sprint': 3
}
# this function returns how many records each course has
def number_of_times(course_name):
    if course_name in tied_times: return tied_times[course_name]
    else: return 2
i = 1
# this loop goes through every row of the table (except the first) and extracts the data
while i < len(rows)-4:
    course_name = ''
    time_150 = 0
    hybrid_150 = False
    main_row = rows[i]
    cols = main_row.find_all('td',recursive=False)


    # the html table is built rather strangely. This code extracts the information from it
    for j, col in enumerate(main_row.find_all('td',recursive=False)):
        if j==0:
            if col.td is not None:
                course_name = col.td.text
        elif j==1:
            time_150 = parse_time(col.text)
            if col.img is not None:
                hybrid_150 = True
        elif j ==2:
            player_150 = col.text
        elif j ==3:
            country_150 = col.img.get_attribute_list('title')[0]
        elif j ==4:
            date_150 = col.text
        elif j ==5:
            duration_150 = col.text
        elif j==6:
            character_150 = col.text
        elif j==7:
            kart_150 = col.text
        elif j==8:
            wheels_150 = col.text
        elif j==9:
            glider_150 = col.text
        elif j==10:
            mouseover = col.img.get_attribute_list('onmouseover')[0]
            splits_150, mushrooms_150, coins_150 = parse_mouseover(mouseover)
    records.append(Record(course_name,150,time_150,player_150,country_150,date_150, duration_150, character_150,kart_150,wheels_150,glider_150,splits_150,mushrooms_150,coins_150,hybrid_150))
    if course_name in tied_times:
        cc_200_row = rows[i+4]
    else:
        cc_200_row = rows[i+3]
    for j, col in enumerate(cc_200_row.find_all('td',recursive=False)):
        if j==0:
            time_200 = parse_time(col.text)
            if col.img is not None:
                hybrid_200 = True
        elif j ==1:
            player_200 = col.text
        elif j ==2:
            country_200 = col.img.get_attribute_list('title')[0]
        elif j ==3:
            date_200 = col.text
        elif j ==4:
            duration_200 = col.text
        elif j==5:
            character_200 = col.text
        elif j==6:
            kart_200 = col.text
        elif j==7:
            wheels_200 = col.text
        elif j==8:
            glider_200 = col.text
        elif j==9:
            mouseover = col.img.get_attribute_list('onmouseover')[0]
            splits_200, mushrooms_200, coins_200 = parse_mouseover(mouseover)
    
    
    records.append(Record(course_name,200,time_200,player_200,country_200,date_200, duration_200, character_200,kart_200,wheels_200,glider_200,splits_200,mushrooms_200,coins_200,hybrid_200))
    
    i += number_of_times(course_name)+2
    
    

In [6]:
# go through and build the dataframe
pd.set_option('display.max_rows',5)
df_rows = []
for record in records:
    s = pd.Series({
        'track': record.track,
        'cc': record.cc,
        'time':record.time,
        'player':record.player,
        'nation':record.nation,
        'date':record.date,
        'duration':record.duration,
        'character':record.character,
        'vehicle':record.vehicle,
        'tires': record.tires,
        'glider': record.glider,
        'splits': record.splits,
        'mushrooms': record.mushroom,
        'coins':record.coin,
        'hybrid_controls_used':record.hybrid_used
    })
    df_rows.append(s)
wr_df = pd.DataFrame(data=df_rows)






wr_df.to_excel('worldrecords.xlsx')
wr_df.to_parquet('worldrecords.parquet')
    


In [7]:
wr_df

,track,cc,time,player,nation,date,duration,character,vehicle,tires,glider,splits,mushrooms,coins,hybrid_controls_used
0,Mario Kart Stadium,150,94.712,Byron,Australia,2022-07-06,126,Roy,Wild Wiggler,Azure Roller,Paper Glider,"[33.047, 30.906, 30.759]","[1, 1, 1]","[8, 2, 0]",True
1,Mario Kart Stadium,200,67.363,Army,France,2022-02-13,269,Wario,Wild Wiggler,Azure Roller,Paper Glider,"[23.521, 22.069, 21.773]","[1, 1, 1]","[7, 3, 0]",True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126,Sky-High Sundae,150,114.905,AK,Japan,2022-10-08,32,Waluigi,Biddybuggy,Azure Roller,Paper Glider,"[39.302, 37.861, 37.742]","[1, 1, 1]","[1, 0, 0]",True
127,Sky-High Sundae,200,79.812,Hikki,Japan,2022-10-05,35,Roy,Wild Wiggler,Leaf Tires,Paper Glider,"[27.645, 26.091, 26.076]","[1, 1, 1]","[1, 0, 0]",True
